# Spotify Music Popularity Capstone - Modeling

Greg Welliver   

In [14]:
# Import relevant libraries and packages.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as plticker
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import seaborn as sns
sns.set()
%matplotlib inline


import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from scipy import stats

from sklearn import linear_model, preprocessing, tree, svm, datasets, metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, auc, mean_squared_error, r2_score, f1_score, log_loss
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder as OHE
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import xgboost
from xgboost import XGBClassifier
import warnings
from subprocess import call
from IPython.display import Image
from datetime import datetime, timedelta, date
import os
import plotly.graph_objects as go
import itertools
from io import StringIO  
import pydotplus


In [3]:
# load the data
X_train = pd.read_csv('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/X_train.csv', index_col =[0])
X_test = pd.read_csv('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/X_test.csv', index_col =[0])
y_train = pd.read_csv('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/y_train.csv', index_col =[0]) 
y_test = pd.read_csv('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/y_test.csv', index_col =[0])
y_test.head()

,Top100
233445,1
120671,1
68604,1
160156,0
52414,1


In [4]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

### Random Forest

In [5]:
# look at available hyperparameters
rf = RandomForestClassifier(random_state = 123)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 123,
 'verbose': 0,
 'warm_start': False}


Scikit-Learn documentation tells us that the most important settings are the number of trees in the forest (n_estimators) and the number of features considered for splitting at each leaf node (max_features)

We will try adjusting the following set of hyperparameters:

- n_estimators = number of trees in the foreset
- max_features = max number of features considered for splitting a node
- max_depth = max number of levels in each decision tree
- min_samples_split = min number of data points placed in a node before the node is split
- min_samples_leaf = min number of data points allowed in a leaf node
- bootstrap = method for sampling data points (with or without replacement)

#### Build Random Forest Model
First, we will train the model with default Random Forest Classifier Hyperparameters.

In [25]:
model = RandomForestClassifier(random_state=123)
model.fit(X_train, y_train.values.ravel())
  
# predict the mode
y_pred = model.predict(X_test)
  
# performance evaluatio metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.83      0.86       378
           1       0.84      0.90      0.87       368

    accuracy                           0.87       746
   macro avg       0.87      0.87      0.87       746
weighted avg       0.87      0.87      0.87       746



In [26]:
print("Classification Report for Training Data")
print(classification_report(y_train, model.predict(X_train)))
print("Classification Report for Test Data")
print(classification_report(y_test, model.predict(X_test)))

Classification Report for Training Data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1485
           1       1.00      1.00      1.00      1495

    accuracy                           1.00      2980
   macro avg       1.00      1.00      1.00      2980
weighted avg       1.00      1.00      1.00      2980

Classification Report for Test Data
              precision    recall  f1-score   support

           0       0.90      0.83      0.86       378
           1       0.84      0.90      0.87       368

    accuracy                           0.87       746
   macro avg       0.87      0.87      0.87       746
weighted avg       0.87      0.87      0.87       746



In [9]:
# set the parameter grid
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

#### Hyperparameter Tuning- GridSearchCV
First, let’s use GridSearchCV to obtain the best parameters for the model. For that, we will pass RandomFoestClassifier() instance to the model and then fit the GridSearchCV using the training data to find the best parameters.

In [10]:
grid_search = GridSearchCV(RandomForestClassifier(random_state=123),
                           param_grid=param_grid)
grid_search.fit(X_train, y_train.values.ravel())
print(grid_search.best_estimator_)

RandomForestClassifier(max_depth=6, max_features=None, max_leaf_nodes=9,
                       n_estimators=150, random_state=123)


#### Update the Model
Now we will update the parameters of the model by those which are obtained by using GridSearchCV.

In [30]:
model_grid = RandomForestClassifier(max_depth=6,
                                    max_leaf_nodes=9,
                                    n_estimators=150, random_state=123)
model_grid.fit(X_train, y_train.values.ravel())
y_pred_grid = model_grid.predict(X_test)

In [20]:
print("Classification Report for Training Data")
print(classification_report(y_train, model_grid.predict(X_train)))
print("Classification Report for Test Data")
print(classification_report(y_test, model_grid.predict(X_test)))

Classification Report for Training Data
              precision    recall  f1-score   support

           0       0.93      0.76      0.84      1485
           1       0.80      0.94      0.86      1495

    accuracy                           0.85      2980
   macro avg       0.86      0.85      0.85      2980
weighted avg       0.86      0.85      0.85      2980

Classification Report for Test Data
              precision    recall  f1-score   support

           0       0.92      0.74      0.82       378
           1       0.77      0.93      0.85       368

    accuracy                           0.83       746
   macro avg       0.85      0.83      0.83       746
weighted avg       0.85      0.83      0.83       746



#### Hyperparameter Tuning- RandomizedSearchCV
Now let’s use RandomizedSearchCV to obtain the best parameters for the model. For that, we will pass RandomFoestClassifier() instance to the model and then fit the RandomizedSearchCV using the training data to find the best parameters.

In [15]:
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=123),
                                   param_grid)
random_search.fit(X_train, y_train.values.ravel())
print(random_search.best_estimator_)

RandomForestClassifier(max_depth=9, max_features=None, max_leaf_nodes=9,
                       n_estimators=50, random_state=123)


#### Update the model
Now we will update the parameters of the model by those which are obtained by using RandomizedSearchCV.

In [28]:
model_random = RandomForestClassifier(max_depth=9,
                                      max_leaf_nodes=9,
                                      n_estimators=50, random_state=123)
model_random.fit(X_train, y_train.values.ravel())
y_pred_rand = model_random.predict(X_test)

In [29]:
print("Classification Report for Training Data")
print(classification_report(y_train, model_random.predict(X_train)))
print("Classification Report for Test Data")
print(classification_report(y_test, model_random.predict(X_test)))

Classification Report for Training Data
              precision    recall  f1-score   support

           0       0.91      0.73      0.81      1485
           1       0.77      0.93      0.85      1495

    accuracy                           0.83      2980
   macro avg       0.84      0.83      0.83      2980
weighted avg       0.84      0.83      0.83      2980

Classification Report for Test Data
              precision    recall  f1-score   support

           0       0.90      0.72      0.80       378
           1       0.76      0.92      0.83       368

    accuracy                           0.82       746
   macro avg       0.83      0.82      0.82       746
weighted avg       0.83      0.82      0.82       746



#### Random Search Version 2

In [31]:
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=123),
                                   param_grid, n_iter = 100, cv = 3, verbose=2, random_state=123, n_jobs = -1)
random_search.fit(X_train, y_train.values.ravel())
print(random_search.best_estimator_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
RandomForestClassifier(max_depth=6, max_features=None, max_leaf_nodes=9,
                       random_state=123)


In [37]:
model_random = RandomForestClassifier(max_depth=6,
                                      max_leaf_nodes=9,
                                      random_state=123)
model_random.fit(X_train, y_train.values.ravel())
y_pred_rand = model_random.predict(X_test)

In [38]:
print("Classification Report for Training Data")
print(classification_report(y_train, model_random.predict(X_train)))
print("Classification Report for Test Data")
print(classification_report(y_test, model_random.predict(X_test)))

Classification Report for Training Data
              precision    recall  f1-score   support

           0       0.92      0.75      0.83      1485
           1       0.79      0.94      0.86      1495

    accuracy                           0.84      2980
   macro avg       0.86      0.84      0.84      2980
weighted avg       0.86      0.84      0.84      2980

Classification Report for Test Data
              precision    recall  f1-score   support

           0       0.91      0.73      0.81       378
           1       0.77      0.93      0.84       368

    accuracy                           0.83       746
   macro avg       0.84      0.83      0.83       746
weighted avg       0.84      0.83      0.83       746



### User Random Hyperparameter Grid

In [ ]:
# we first need to create a parameter grid to sample from during fitting

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 5, stop = 200, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(3, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
# instantiate the random search and fit it
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=123, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train.values.ravel())



The most important arguments in RandomizedSearchCV are n_iter, which controls the number of different combinations to try, and cv which is the number of folds to use for cross validation (we use 100 and 3 respectively). More iterations will cover a wider search space and more cv folds reduces the chances of overfitting, but raising each will increase the run time. Machine learning is a field of trade-offs, and performance vs time is one of the most fundamental.

In [ ]:
# We can view the best parameters from fitting the random search:
rf_random.best_params_

# From these results, we should be able to narrow the range of values for each hyperparameter.

### Evaluate Random Search
To determine if random search yielded a better model, we compare the base model with the best random search model.

def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestClassifier(n_estimators = 10, random_state = 123)
base_model.fit(X_train, y_train.values.ravel())
base_accuracy = evaluate(base_model, X_test, y_test)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_features, test_labels)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))



### Grid Search with Cross Validation

Random search allowed us to narrow down the range for each hyperparameter. Now that we know where to concentrate our search, we can explicitly specify every combination of settings to try. We do this with GridSearchCV, a method that, instead of sampling randomly from a distribution, evaluates all combinations we define. To use Grid Search, we make another grid based on the best values provided by random search:

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [40, 50, 60, 70],
    'max_features': [2, 3],
    'min_samplems_leaf': [2, 3],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [50, 100, 200, 5000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

We can fit the model, display the best hyperparameters, and evaluate performance:

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

# {'bootstrap': True,
#  'max_depth': 80,
#  'max_features': 3,
#  'min_samples_leaf': 5,
#  'min_samples_split': 12,
#  'n_estimators': 100}

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

# Model Performance
# Average Error: 3.6561 degrees.
# Accuracy = 93.83%.

print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

# Improvement of 0.50%.

In [ ]:
RandomForestClassifier().get_params().keys()

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=3, random_state = 123, n_jobs=-1)
model_res = rf_clf.fit(X_train, y_train.values.ravel())
print('Accuracy on training set:',rf_clf.score(X_train,y_train))
print('Accuracy on test set:',rf_clf.score(X_test,y_test))
y_pred = model_res.predict(X_test)
y_pred_prob = model_res.predict_proba(X_test)
lr_probs = y_pred_prob[:,1]
ac = accuracy_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

print('Random Forest: Accuracy=%.3f' % (ac))

print('Random Forest: f1-score=%.3f' % (f1))

In [ ]:
# let's look at classification report
print("Classification Report for Training Data")
print(classification_report(y_train, rf_clf.predict(X_train)))
print("Classification Report for Test Data")
print(classification_report(y_test, rf_clf.predict(X_test)))

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=300, random_state = 123, n_jobs=-1)
model_res = rf_clf.fit(X_train, y_train.values.ravel())
print('Accuracy on training set:',rf_clf.score(X_train,y_train))
print('Accuracy on test set:',rf_clf.score(X_test,y_test))
y_pred = model_res.predict(X_test)
y_pred_prob = model_res.predict_proba(X_test)
lr_probs = y_pred_prob[:,1]
ac = accuracy_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

print('Random Forest: Accuracy=%.3f' % (ac))

print('Random Forest: f1-score=%.3f' % (f1))


In [ ]:
# let's look at classification report
print("Classification Report for Training Data")
print(classification_report(y_train, rf_clf.predict(X_train)))
print("Classification Report for Test Data")
print(classification_report(y_test, rf_clf.predict(X_test)))

In [ ]:
# look at confusion matrix
class_names=['0','1'] # name  of classes

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.grid(False)
    #plt.rcParams.update({'font.size': 7})


# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')
#plt.savefig('figures/RF_cm_multi_class.png')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
#plt.savefig('figures/RF_cm_proportion_multi_class.png', bbox_inches="tight")
#plt.rcParams.update({'font.size': 7})
plt.show()


In [ ]:
# plot feature importances
feature_importance = rf_clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
sorted_idx = np.argsort(feature_importance)[:30]

pos = np.arange(sorted_idx.shape[0]) + .5
print(pos.size)
sorted_idx.size
plt.figure(figsize=(10,10))
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, X.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()


#### QUESTIONS: 
- should i try someting here to optimize the number of estimators?
- should accuracy of training data be 1.0?
- in most of my calculations, accuracy and F1 score were either the same, or very close. is this expected?

#### write to CSV
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)

#### write to parquet
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled_pq2.parquet')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_parquet(filepath, 
              engine = "pyarrow", 
              compression = None)

df = pd.read_parquet('/Users/gregwelliver/Desktop/springboard_files/Music-Popularity-Capstone-Repo/Data/df_data_scaled_pq.parquet')